<img src="https://juliacomputing.com/assets/img/new/JuliaDB_logo2.svg" width=200>

<img src ="http://pluspng.com/img-png/free-png-plus-sign-plus-icon-512.png" width=50>

<img src = "https://user-images.githubusercontent.com/8075494/39086572-80874e5a-4561-11e8-9a05-e52b21a3e580.png" width=400>

# Dr. Josh Day
# `@joshday`
### https://github.com/joshday/Talks/ODSC2018_JuliaDB_OnlineStats

---

# Why Julia?

- https://julialang.org/blog/2012/02/why-we-created-julia
- The dream:
    - **Be (at least) as nice as Python/R/Matlab**
    - **Be as fast as C/Fortran**
    
---

###  JuliaDB is a distributed analytical database implemented in 100% Julia
- Load multi-file datasets
- All user-defined functions are compiled
- You can store any data type
- You can turn on parallel computing with the flip of a switch
- Limited (but growing) out-of-core support

### OnlineStats is collection of parallelizable single-pass algorithms for statistics
- You can optionally give recent observations higher influence over models/statistics

<img src = "https://user-images.githubusercontent.com/8075494/38169834-e15b1b32-3542-11e8-8789-e6f6e3296e8e.gif" width=400>

### JuliaDB now fully integrates OnlineStats! 



---

# Jump Into Example 

## (Subset of) "Huge Stock Market Dataset"

- Source: https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
    - Each stock's OHLCV data in its own file

### First, examine our data files

In [4]:
;ls stocks

aapl.us.txt
amzn.us.txt
dis.us.txt
googl.us.txt
ibm.us.txt
msft.us.txt
nflx.us.txt
tsla.us.txt


In [5]:
;head stocks/aapl.us.txt

Date,Open,High,Low,Close,Volume,OpenInt
1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0
1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0
1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0
1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0
1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0
1984-09-14,0.44052,0.45589,0.44052,0.44566,68847968,0
1984-09-17,0.45718,0.46357,0.45718,0.45718,53755262,0
1984-09-18,0.45718,0.46103,0.44052,0.44052,27136886,0
1984-09-19,0.44052,0.44566,0.43157,0.43157,29641922,0


## Load packages, force tables to print data

In [6]:
using JuliaDB, OnlineStats

# Force table to print out
IndexedTables.set_show_compact!(false);

### A Table is one of the main data structures in JuliaDB

#### `loadtable` will load all CSV files in the directory

In [7]:
t = loadtable("stocks")

Table with 56023 rows, 7 columns:
Date        Open     High     Low      Close    Volume    OpenInt
─────────────────────────────────────────────────────────────────
1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030  0
1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532  0
1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199  0
1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801  0
1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062  0
1984-09-14  0.44052  0.45589  0.44052  0.44566  68847968  0
1984-09-17  0.45718  0.46357  0.45718  0.45718  53755262  0
1984-09-18  0.45718  0.46103  0.44052  0.44052  27136886  0
1984-09-19  0.44052  0.44566  0.43157  0.43157  29641922  0
1984-09-20  0.43286  0.43668  0.43286  0.43286  18453585  0
1984-09-21  0.43286  0.44566  0.42388  0.42902  27842780  0
1984-09-24  0.42902  0.43157  0.42516  0.42516  22033109  0
⋮
2017-10-27  319.75   324.59   316.66   320.87   6970118   0
2017-10-30  319.18   323.78   317.25   320.08   4254

### We can do better than that

- We are missing stock ticker!
- `OpenInt` seems like a useless variable
- We can also **Index** (sort) the data

In [8]:
t = loadtable("stocks"; filenamecol = :Ticker, datacols = 3:7, indexcols=1:2)

Table with 56023 rows, 7 columns:
Ticker         Date        Open     High     Low      Close    Volume
───────────────────────────────────────────────────────────────────────
"aapl.us.txt"  1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030
"aapl.us.txt"  1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532
"aapl.us.txt"  1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199
"aapl.us.txt"  1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801
"aapl.us.txt"  1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062
"aapl.us.txt"  1984-09-14  0.44052  0.45589  0.44052  0.44566  68847968
"aapl.us.txt"  1984-09-17  0.45718  0.46357  0.45718  0.45718  53755262
"aapl.us.txt"  1984-09-18  0.45718  0.46103  0.44052  0.44052  27136886
"aapl.us.txt"  1984-09-19  0.44052  0.44566  0.43157  0.43157  29641922
"aapl.us.txt"  1984-09-20  0.43286  0.43668  0.43286  0.43286  18453585
"aapl.us.txt"  1984-09-21  0.43286  0.44566  0.42388  0.42902  27842780
"aapl.us.txt"  1984-09-24  0.429

## NDSparse is the other main data structure in JuliaDB

In [9]:
nd = loadndsparse("stocks", filenamecol = :Ticker, datacols = 3:7, indexcols = 1:2)

2-d NDSparse with 56023 values (5 field named tuples):
Ticker         Date       │ Open     High     Low      Close    Volume
──────────────────────────┼─────────────────────────────────────────────
"aapl.us.txt"  1984-09-07 │ 0.42388  0.42902  0.41874  0.42388  23220030
"aapl.us.txt"  1984-09-10 │ 0.42388  0.42516  0.41366  0.42134  18022532
"aapl.us.txt"  1984-09-11 │ 0.42516  0.43668  0.42516  0.42902  42498199
"aapl.us.txt"  1984-09-12 │ 0.42902  0.43157  0.41618  0.41618  37125801
"aapl.us.txt"  1984-09-13 │ 0.43927  0.44052  0.43927  0.43927  57822062
"aapl.us.txt"  1984-09-14 │ 0.44052  0.45589  0.44052  0.44566  68847968
"aapl.us.txt"  1984-09-17 │ 0.45718  0.46357  0.45718  0.45718  53755262
"aapl.us.txt"  1984-09-18 │ 0.45718  0.46103  0.44052  0.44052  27136886
"aapl.us.txt"  1984-09-19 │ 0.44052  0.44566  0.43157  0.43157  29641922
"aapl.us.txt"  1984-09-20 │ 0.43286  0.43668  0.43286  0.43286  18453585
"aapl.us.txt"  1984-09-21 │ 0.43286  0.44566  0.42388  0.42902  2784278

### Table and NDSparse print differently, but how are they different?

#### A Table accesses data by *row*

In [10]:
t[1]

(Ticker = "aapl.us.txt", Date = 1984-09-07, Open = 0.42388, High = 0.42902, Low = 0.41874, Close = 0.42388, Volume = 23220030)

In [11]:
t[1].Volume

23220030

#### An NDSparse accesses data by *indexes*

- API mirrors a sparse array (but with arbitrary indexes)

In [12]:
nd["aapl.us.txt", Date(1984,9,7)]

(Open = 0.42388, High = 0.42902, Low = 0.41874, Close = 0.42388, Volume = 23220030)

In [13]:
nd["aapl.us.txt", Date(1984, 9, 7)].Volume

23220030

# Why NDSparse?

## What was Apple's closing price on February 10, 1986?

- A Table requires a query

In [14]:
filter(r -> r.Date == Date(1986,2,10) && r.Ticker == "aapl.us.txt", t)

Table with 1 rows, 7 columns:
Ticker         Date        Open     High     Low      Close    Volume
───────────────────────────────────────────────────────────────────────
"aapl.us.txt"  1986-02-10  0.38289  0.39186  0.37906  0.38164  31191161

- With NDSparse, this is just `getindex`

In [15]:
nd["aapl.us.txt", Date(1986,2,10)]

(Open = 0.38289, High = 0.39186, Low = 0.37906, Close = 0.38164, Volume = 31191161)

# Selectors

### Selectors are powerful ways to select and manipulate data

1. `Integer`: column at position
2. `Symbol`: column by name
3. `Array`: itself (useful for adding columns)
4. `Pair{Selection => Function}`: function mapped to selection
5. `Tuple` of selections: table of each selection

### Selectors show up in many places (everything in green)

<code>select(t, <span style="color: green">which</span>)
map(f, t; <span style="color: green">select</span>)
reduce(f, t; <span style="color: green">select</span>)
filter(f, t; <span style="color: green">select</span>)
groupby(f, t, <span style="color: green">by</span>; <span style="color: green">select</span>)
groupreduce(f, t, <span style="color: green">by</span>; <span style="color: green">select</span>)
join(f, l, r; how, <span style="color: green">lkey</span>, <span style="color: green">rkey</span>, <span style="color: green">lselect</span>, <span style="color: green">rselect</span>)
groupjoin(f, l, r; how, <span style="color: green">lkey</span>, <span style="color: green">rkey</span>, <span style="color: green">lselect</span>, <span style="color: green">rselect</span>)
</code>

### Using OnlineStats, let's get the average (in millions) Volume, grouped by Ticker

In [61]:
@time groupreduce(Mean(), t, :Ticker, select = :Volume => x -> x / 10^6)

  0.036838 seconds (15.91 k allocations: 1.270 MiB)


Table with 8 rows, 2 columns:
Ticker          Mean
─────────────────────────────────────────────
"aapl.us.txt"   Mean: n=8364 | value=106.642
"amzn.us.txt"   Mean: n=5153 | value=7.83732
"dis.us.txt"    Mean: n=12072 | value=7.10866
"googl.us.txt"  Mean: n=3333 | value=7.95173
"ibm.us.txt"    Mean: n=14059 | value=5.78297
"msft.us.txt"   Mean: n=7983 | value=79.458
"nflx.us.txt"   Mean: n=3201 | value=19.5311
"tsla.us.txt"   Mean: n=1858 | value=4.41651

### Alternatively, we can let OnlineStats perform the grouping

In [33]:
g = reduce(GroupBy{String}(Mean()), t, select = (:Ticker, :Volume))

GroupBy{String,Mean{EqualWeight}}
  ├── aapl.us.txt: Mean: n=8364 | value=1.06642e8
  ├── amzn.us.txt: Mean: n=5153 | value=7.83732e6
  ├── dis.us.txt: Mean: n=12072 | value=7.10866e6
  ├── googl.us.txt: Mean: n=3333 | value=7.95173e6
  ├── ibm.us.txt: Mean: n=14059 | value=5.78297e6
  ├── msft.us.txt: Mean: n=7983 | value=7.9458e7
  ├── nflx.us.txt: Mean: n=3201 | value=1.95311e7
  └── tsla.us.txt: Mean: n=1858 | value=4.41651e6

# Many stats can be plotted

## GroupBy

In [36]:
using Plots
plotly()

scatter(g, title = "Average Volume")

## Hist

In [37]:
plot(fit!(Hist(50), randn(10^6)))

## CountMap

In [43]:
y = rand([1,2,2,3,3,3,4,4,4,4], 10^6)

@time o = fit!(CountMap(Int), y)

plot(o)

  0.046924 seconds (12 allocations: 784 bytes)


## Partition and IndexedPartition

- Unique structure for plotting a **summary** of a large dataset

In [47]:
y = cumsum(randn(10^6)) + 100randn(10^6)

o = Partition(Hist(10))

fit!(o, y)

plot(o, xlab = "Nobs")

In [50]:
x = randn(10^7)
y = x + randn(10^7)

o = fit!(IndexedPartition(Float64, Hist(10)), table(x,y))

plot(o, ylab = "Y", xlab = "X")